# Resampling Methods (MACS 30100)
### by [Benjamin Soltoff](http://www.bensoltoff.com/) and [Richard W. Evans](https://sites.google.com/site/rickecon/), February 2018
The code in this Jupyter notebook was written using Python 3.6. It uses data files `?`, and ? image files in the `images` folder in the same directory as this notebook. For the code to run properly, you will either need to have access to the internet or you should have the data file in the same folder as the Jupyter notebook file. Otherwise, you will have to change the respective lines of the code that read in the data to reflect the location of that data. Some of this content was taken from Dr. Benjamin Soltoff's notes [here](http://cfss.uchicago.edu/persp006_resampling.html).

Resampling methods are a way to test the sensitivity of statistical results to estimation using a different sample. It is often too difficult or too expensive to draw a new sample from the population. Resampling methods take advantage of the training-set test-set paradigm to evaluate the sensitivity of estimates to sample variance. The two main classes of resampling methods are

1. Cross validation
2. Bootstrapping

## 1. Cross validation

### 1.1. Validation set approach
This is the approach that we have already studied in the [classifiers 1 notebook](https://github.com/UC-MACSS/persp-model_W18/blob/master/Notebooks/Classfcn1/KKNlogitLDA.ipynb).

1. Partition the data into a training set and a test set.
2. Estimate the model using the training set.
3. Evaluate the fit or predictive accuracy on the test set.

### 1.2. Leave-one-out cross validation
Leave-one-out cross validation (LOOCV) is an approach in which the model is assessed using $N$ different training sets and test sets of a specific size. Let the data have $N$ observations. LOOCV is to choose a training set with $N-1$ observations, such that the test set only has one observation $y_i$. Repeat this $N$ with a slightly different training set such that each data point is the test set in exactly one of these sebsets.

In this case, the mean squared error MSE has no summation because there is only one observation in the test set.

$$ MSE_i = (y_j - \hat{y}_j)^2 $$

The LOOCV estimate for the test MSE is the average of these $N$ test error estimates.

$$ CV_{loo} = \frac{1}{N}\sum_{i=1}^N MSE_i $$

### 1.3. k-fold cross validation
$k$-fold cross validation is a method in which the dataset is randomly divided into $k$ groups (folds). Define a test set of the model as the $k$th fold. For each test set $k$, the model is estimated on the data from the other $k-1$ folds. Let the number of observations in the $k$th fold be $N_k$, and let $\mathcal{K}$ be the set of observations in the $k$th fold. The $MSE_k$ of the $k$th fold is:

$$ MSE_k = \frac{1}{N_k}\sum_{i\in\mathcal{K}}(y_i - \hat{y}_i)^2 $$

Then the $k$-fold estimate for the test MSE is the average of these $k$ test error estimates.

$$ CV_{kf} = \frac{1}{k}\sum_{j=1}^k MSE_j $$

LOOCV is a special case of $k$-fold cross validation in which $k=N$.

### 1.4. Bias versus variance
Note that the LOOCV method has low bias (estimated on large number of date) but high variance (errors are based on one draw). In contrast, the $k$-fold method has more bias (estimated with less data) but lower variance. Each test set has more observations.

* $k$-fold cross validation can often provide more accurate estimates of the test error rate.
* $k$-fold is less computationally intensive
* LOOCV has the least bias
* LOOCV is the most computationally expensive

## 2. Bootstrapping
This name comes from the expression "to pull oneself up by ones own bootstraps." In a way similar to the cross validation methods of the last section, we can use *the bootstrap* to quantify the undertainty associated with a given estimator, learning model, or method. In the econometrics and statistics literature, this often shows up as "bootstrapped standard errors". Bootstrapping is valuable because it is so widely applicable to a range of models.

1. Randomly draw $S$ datasets of size $N_S$ with replacement. Define each training set of observations as $\mathcal{K}_s$ and each corresponding test set as $\mathcal{-K}_{s}$.
2. Calculate the MSE for each test set $\mathcal{-K}_{s}$

The bootstrap estimate for the test MSE is the average MSE from each random test set.

$$ CV_{boot} = \frac{1}{S}\sum_{s=1}^S MSE_s $$